In [4]:
import tensorflow as tf
from keras.applications import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import cv2 
import numpy as np

In [9]:
# Set the input image size
input_size = (512, 512)

In [54]:

# Load the Inception v3 model without the top classification layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(input_size[0], input_size[1], 3))


In [55]:
# Freeze the pre-trained layers
for layer in base_model.layers[:200]:
    layer.trainable = False


In [56]:
# Add a new classification layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # 4 output units for the tumor classes


In [57]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
def pre_proc_fn(img):    
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image = cv2.GaussianBlur(src=image,ksize=(5,5),sigmaX=0,sigmaY=0)
    ##image = cv2.equalizeHist(src=image, )
    image = np.expand_dims(image, axis=-1)
    return image

In [15]:

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    preprocessing_function=pre_proc_fn
)

In [5]:
test_datagen = ImageDataGenerator(
    rescale = 1./255,
)

In [6]:
 
path = 'C:\\Users\\Abhijith_Kumble\\Desktop\\IEEE\\brain_Tumor_final\\brain_tumor'

# Set the paths to your training and testing data
train_data_dir = path+'\\Training'
test_data_dir = path + '\\Testing'




In [16]:
image_size = (112,112)
# Set the batch size
batch_size = 32

# Generate training and testing data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = 'rgb'
    
)


Found 5712 images belonging to 4 classes.


In [24]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = 'rgb',
    shuffle=False,
    
)


Found 1311 images belonging to 4 classes.


In [ ]:

# Set the number of training and testing samples
num_train_samples = train_generator.samples
num_test_samples = test_generator.samples

# Set the number of training epochs
num_epochs = 10

In [63]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_376 (Conv2D)            (None, 255, 255, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_376 (Batch  (None, 255, 255, 32  96         ['conv2d_376[0][0]']             
 Normalization)                 )                                                           

In [64]:
print(len(model.layers))

314


In [65]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=num_epochs,

)

Epoch 1/10
178/178 [==============================] - 1347s 8s/step - loss: 0.3257 - accuracy: 0.8877
Epoch 2/10
178/178 [==============================] - 1325s 7s/step - loss: 0.1377 - accuracy: 0.9540
Epoch 3/10
178/178 [==============================] - 1354s 8s/step - loss: 0.1013 - accuracy: 0.9655
Epoch 4/10
178/178 [==============================] - 1504s 8s/step - loss: 0.0836 - accuracy: 0.9715
Epoch 5/10
178/178 [==============================] - 1745s 10s/step - loss: 0.0769 - accuracy: 0.9750
Epoch 6/10
178/178 [==============================] - 3380s 19s/step - loss: 0.0635 - accuracy: 0.9776
Epoch 7/10
178/178 [==============================] - 1299s 7s/step - loss: 0.0466 - accuracy: 0.9861
Epoch 8/10
178/178 [==============================] - 1305s 7s/step - loss: 0.0364 - accuracy: 0.9875
Epoch 9/10
178/178 [==============================] - 1319s 7s/step - loss: 0.0493 - accuracy: 0.9833
Epoch 10/10
178/178 [==============================] - 1288s 7s/step - loss: 0.0

In [66]:
model.save('inception_v3_ver2.2.h5')

In [18]:
model = tf.keras.models.load_model('inception_v3_ver2.2.h5')

In [25]:
predictions = model.predict(test_generator)

41/41 [==============================] - 151s 4s/step


In [26]:
print(predictions)

[[1.00000000e+00 1.37121923e-08 2.68062041e-14 4.63023092e-10]
 [9.99817431e-01 1.15903771e-04 3.18452322e-08 6.67214626e-05]
 [9.99999881e-01 6.68896405e-08 1.27009833e-10 1.08283418e-08]
 ...
 [4.59637189e-10 1.01872721e-09 1.29383046e-10 1.00000000e+00]
 [1.01467564e-10 3.45680637e-11 9.37139797e-13 1.00000000e+00]
 [7.63152930e-10 3.11529746e-10 7.09135042e-11 1.00000000e+00]]


In [27]:
np.save('predictions.npy',predictions)

In [28]:
predicted_labels = predictions.argmax(axis=1)

In [29]:
from sklearn.metrics import accuracy_score,recall_score,f1_score,confusion_matrix

In [32]:
true_labels = np.array(test_generator.labels)

In [34]:
accuracy = accuracy_score(true_labels,predicted_labels, )

In [37]:
f1 = f1_score(true_labels,predicted_labels,average='weighted')

In [38]:
recall = recall_score(true_labels,predicted_labels,average='weighted')

In [45]:
print('accuracy  =' , round(accuracy*100,4))
print('f1        =' , round(f1,6))
print('recall    =' , round(recall,6))


accuracy  = 96.8726
f1        = 0.968716
recall    = 0.968726


In [43]:
confusion_matrix(true_labels,predicted_labels)

array([[284,  10,   0,   6],
       [  1, 282,   2,  21],
       [  0,   0, 405,   0],
       [  0,   1,   0, 299]], dtype=int64)